# 📊 Crypto IDX Perfect Entry Signals (1m)
यह notebook BTC + ETH analysis करके **Crypto IDX signals** generate करेगा।

👉 Steps:
1. नीचे Config में अपना **TELEGRAM_TOKEN** और **CHAT_ID** डालो।
2. सभी cells run करो।
3. हर 1 minute में signal तुम्हें Telegram पर मिलेगा 🚀

In [ ]:
# ✅ CONFIG
TELEGRAM_TOKEN = "PASTE_YOUR_TOKEN_HERE"   # BotFather से मिला token डालो
CHAT_ID = "PASTE_YOUR_CHATID_HERE"         # Helper से निकाला हुआ Chat ID डालो

In [ ]:
!pip install ccxt pandas ta requests

In [ ]:
import ccxt
import pandas as pd
import requests
import time
from ta.trend import SMAIndicator, MACD
from ta.momentum import RSIIndicator

# Telegram sender
def send_telegram(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    payload = {"chat_id": CHAT_ID, "text": message}
    try:
        requests.post(url, data=payload)
    except Exception as e:
        print("Telegram Error:", e)

# Analysis function
def analyze_symbol(symbol="BTC/USDT", timeframe="1m", limit=200):
    exchange = ccxt.binance()
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

    df = pd.DataFrame(ohlcv, columns=['time','open','high','low','close','volume'])
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    df.set_index('time', inplace=True)

    # Indicators
    df['sma_fast'] = SMAIndicator(df['close'], window=9).sma_indicator()
    df['sma_slow'] = SMAIndicator(df['close'], window=21).sma_indicator()
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    macd = MACD(df['close'])
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()

    # Default
    df['signal'] = 'WAIT'
    df['entry'] = None
    df['sl'] = None
    df['tp'] = None

    # BUY setup
    df.loc[(df['sma_fast'] > df['sma_slow']) &
           (df['rsi'] < 35) &
           (df['macd'] > df['macd_signal']) &
           (df['macd'] > 0), 'signal'] = 'BUY'

    # SELL setup
    df.loc[(df['sma_fast'] < df['sma_slow']) &
           (df['rsi'] > 65) &
           (df['macd'] < df['macd_signal']) &
           (df['macd'] < 0), 'signal'] = 'SELL'

    last = df.tail(1).copy()
    entry = last['close'].values[0]

    if last['signal'].values[0] == "BUY":
        sl = df['low'][-5:].min()
        tp = entry + 2 * (entry - sl)
        last['entry'] = entry
        last['sl'] = sl
        last['tp'] = tp

    elif last['signal'].values[0] == "SELL":
        sl = df['high'][-5:].max()
        tp = entry - 2 * (sl - entry)
        last['entry'] = entry
        last['sl'] = sl
        last['tp'] = tp

    return last[['close','rsi','macd','macd_signal','signal','entry','sl','tp']]

def final_signal(btc, eth):
    btc_sig = btc['signal'].values[0]
    eth_sig = eth['signal'].values[0]

    if btc_sig == "BUY" and eth_sig == "BUY":
        return "BUY"
    elif btc_sig == "SELL" and eth_sig == "SELL":
        return "SELL"
    else:
        return "WAIT"

In [ ]:
# Main loop → हर 1 minute में signal भेजेगा
while True:
    try:
        btc = analyze_symbol("BTC/USDT")
        eth = analyze_symbol("ETH/USDT")

        final = final_signal(btc, eth)
        message = "=== CRYPTO IDX Perfect Entry (1m) ===\n"
        message += f"Final Signal: {final}\n\n"

        # BTC
        message += f"BTC: {btc['signal'].values[0]} | Close={btc['close'].values[0]:.2f} | RSI={btc['rsi'].values[0]:.1f}\n"
        if btc['signal'].values[0] in ["BUY","SELL"]:
            message += f"Entry={btc['entry'].values[0]:.2f}, SL={btc['sl'].values[0]:.2f}, TP={btc['tp'].values[0]:.2f}\n"

        # ETH
        message += f"\nETH: {eth['signal'].values[0]} | Close={eth['close'].values[0]:.2f} | RSI={eth['rsi'].values[0]:.1f}\n"
        if eth['signal'].values[0] in ["BUY","SELL"]:
            message += f"Entry={eth['entry'].values[0]:.2f}, SL={eth['sl'].values[0]:.2f}, TP={eth['tp'].values[0]:.2f}\n"

        print(message)
        if final in ["BUY","SELL"]:
            send_telegram(message)

    except Exception as e:
        print("Error:", e)

    time.sleep(60)